# AWS SDK

The python SKD for AWS is the `boto3` package. This page covers the details of working with the `boto3` package.

In [ ]:
import moto
import boto3
from pprint import pprint

## Moto

[Moto](https://docs.getmoto.org/en/latest/) enables the behaviour of AWS services to be mocked.

---

The following code creates an `s3` client and attemnts to list available buckets.

In [ ]:
s3 = boto3.client("s3", region_name="us-west-2")

try:
    s3.list_buckets()
except Exception as e:
    print(e)

Unable to locate credentials


The result there is an error message indicating that there are no AWS credentials have been provided.

The following cell uses the same `boto3` code wrapped by the `moto` mock.

In [ ]:
mock = moto.mock_aws()
mock.start()

s3 = boto3.client("s3", region_name="us-west-2")
pprint(s3.list_buckets())

mock.stop()

{'Buckets': [],
 'Owner': {'DisplayName': 'webfile', 'ID': 'bcaf1ffd86f41161ca5fb16fd081034f'},
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/xml',
                                      'x-amzn-requestid': 'tzN0rrlVDe1F4epGGP0GictXAyeEdVvaFkcoMC97KHZ3sZqlrJ8p'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'tzN0rrlVDe1F4epGGP0GictXAyeEdVvaFkcoMC97KHZ3sZqlrJ8p',
                      'RetryAttempts': 0}}


The result the is some kind of AWS answer.

---

The following cell shows an example of creating a bucket (`some-bucket`), listing it and deleteting it. Moto somehow retains information about the behaviour of the bucket.

In [ ]:
with moto.mock_aws():
    region = "us-west-2"
    bucket_name = "some-bucket"

    s3 = boto3.client("s3", region_name=region)

    location = {'LocationConstraint': region}
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration=location
    )

    pprint(s3.list_buckets()["Buckets"])

    s3.delete_bucket(Bucket=bucket_name)

[{'CreationDate': datetime.datetime(2025, 10, 20, 12, 8, 15, tzinfo=tzutc()),
  'Name': 'some-bucket'}]


## Services names

Boto provides access to the services thorugh its abstractions: `client`, `resources` etc. You usually have to specify to which service you want to access using a special shortcut. You can get a list of shortcuts from the `boto3.session.Session().get_available_services()` method. 

As there is a huge amount of services awailable in AWS, the following table lists the most imporant for me with a short description:

| Group      | Shortcut                          | Description                                                                    |
|------------|---------------------------------- |--------------------------------------------------------------------------------|
|            | `s3`                              | S3 storage                                                                     |
|            | `iam`                             | Identity and Access Management - managing users and permissions                |
|            | `dynamodb`                        | Dynamo database                                                                |
|            | `lambda`                          | Lambda functions                                                               |
| bedrock    | `bedrock`                         | Core Amazon Bedrock service for managing foundation models and configurations. |
|            | `bedrock-runtime`                 | Runtime APIs for invoking and generating responses from Bedrock models.        |
|            | `bedrock-agent`                   | Control APIs for creating and managing Bedrock AI agents and their resources.  |
|            | `bedrock-agent-runtime`           | Runtime APIs for invoking Bedrock agents and retrieving their outputs.         |
|            | `bedrock-agentcore`               | Platform for building and running advanced AI agents with memory and tools.    |
|            | `bedrock-agentcore-control`       | Control-plane APIs for configuring AgentCore components and deployments.       |
|            | `bedrock-data-automation`         | Service for automating data processing and transformation workflows.           |
|            | `bedrock-data-automation-runtime` | Runtime APIs for executing and monitoring data automation tasks.               |
| openserach | `opensearch`                      | Opensearch database.                                                           |
|            | `opensearchserverless`            | Serverless opensearch databse.                                                 |

Check the [Available Services](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/index.html) page in the boto3 documentation for description of the services.

---

The following cell gets a list of the available services prints their amount and prints a few of them.

In [9]:
with moto.mock_aws():
    session = boto3.session.Session()
    services = session.get_available_services()

print(len(services))
for s in services[:20]:
    print(s)

413
accessanalyzer
account
acm
acm-pca
aiops
amp
amplify
amplifybackend
amplifyuibuilder
apigateway
apigatewaymanagementapi
apigatewayv2
appconfig
appconfigdata
appfabric
appflow
appintegrations
application-autoscaling
application-insights
application-signals


## Client

The [client](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/clients.html) provides a low-level, 1:1 access to the AWS services.

Create client through `boto3.client` method.